In [6]:
import torch
from torchvision import transforms
from torchvision.datasets import CocoDetection
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import fiftyone
import json
import cv2
import os
import shutil

## Load data

In [ ]:
# Load Training dataset
dataset_train = fiftyone.zoo.load_zoo_dataset(
    "coco-2017",
    split="train",
    label_types=["detections"],
    only_matching=True,
    classes=["person","bicycle","car","motorcycle", "airplane","bus","train","truck","boat","traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat","dog","horse"]
    #max_samples=50,
)

# Export Training dataset

dataset_train.export(
    export_dir="/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/train",
    dataset_type=fiftyone.types.COCODetectionDataset,
    label_field="ground_truth"
)

In [ ]:

# Load Validation dataset
dataset_val = fiftyone.zoo.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    only_matching=True,
    classes=["person","bicycle","car","motorcycle", "airplane","bus","train","truck","boat","traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat","dog","horse"]
    #max_samples=50,
)

# Export Validation dataset

dataset_val.export(
    export_dir="/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset/val",
    dataset_type=fiftyone.types.COCODetectionDataset,
    label_field="ground_truth"
)

## Bring Data into YOLO format

In [12]:
# Utils functions

def read_json_file(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data


In [13]:
# Define pathes
input_path = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset'
output_path = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset_yolo'

# json anotation: dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
train_labels = read_json_file(os.path.join(input_path,'train/labels.json'))
val_labels = read_json_file(os.path.join(input_path,'val/labels.json'))


In [ ]:
# processing images

file_names = []

def load_images_from_folder(folder,type = 'train'):
  count = 0
  for filename in os.listdir(folder):
        source = os.path.join(folder,filename)
        destination = f"{output_path}/{type}_images/img{count}.jpg"

        try:
            shutil.copy(source, destination)
        # If source and destination are same
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        file_names.append(filename)
        count += 1

load_images_from_folder(os.path.join(input_path,'val/images'),type = 'val')

## Load Data to Dataloader

a) Labels format: one *.txt file per image

- One row per object
- Each row is class x_center y_center width height format.
- Box coordinates must be in normalized xywh format (from 0 - 1). If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
- Class numbers are zero-indexed (start from 0).


b)

In [37]:
dataset_dir = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset'
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset_val = CocoDetection(root = os.path.join(dataset_dir,'val/data'),annFile = os.path.join(dataset_dir,'val/labels.json'),transform=transform)

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


In [70]:
img, label = dataset_val[3]
label

[{'id': 27,
  'image_id': 4,
  'category_id': 62,
  'bbox': [408.03, 172.04000000000002, 19.38, 16.53],
  'area': 320.3514,
  'iscrowd': 0,
  'supercategory': 'sports'},
 {'id': 28,
  'image_id': 4,
  'category_id': 49,
  'bbox': [145.26, 100.67, 291.95, 457.35],
  'area': 133523.3325,
  'iscrowd': 0,
  'supercategory': 'person'},
 {'id': 29,
  'image_id': 4,
  'category_id': 49,
  'bbox': [163.73, 126.41999999999999, 265.69, 480.4],
  'area': 127637.476,
  'iscrowd': 0,
  'supercategory': 'person'},
 {'id': 30,
  'image_id': 4,
  'category_id': 5,
  'bbox': [368.64, 157.25, 57.45, 45.78],
  'area': 2630.061,
  'iscrowd': 0,
  'supercategory': 'sports'}]

## YOLO Model

In [ ]:
from ultralytics import YOLO
# Load Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)  # yolov5n - yolov5x6 or custom


In [56]:
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

image 1/1: 720x1280 2 persons, 2 ties
Speed: 2414.4ms pre-process, 71.7ms inference, 1.1ms NMS per image at shape (1, 3, 384, 640)


In [13]:
# Set the root directory where you want to download the COCO dataset
root_dir = r'/Users/daniil.yefimov/Desktop/Study/Uni Linz/Autonomus Vehicle/1. Perceptron Project/train2017'
ann_dir = r'/Users/daniil.yefimov/Desktop/Study/Uni Linz/Autonomus Vehicle/1. Perceptron Project/annotations/instances_train2017.json'
# Define transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Download and create the COCO dataset
coco_dataset = CocoDetection(root=root_dir, annFile=ann_dir, transform=transform)




loading annotations into memory...
Done (t=13.91s)
creating index...
index created!


#### from COCO to Yolo format

In [90]:
def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None
    

def get_img(filename):
  for img in data['images']:
    if img['file_name'] == filename:
      return img

In [91]:
data['images'][0]

{'id': 1,
 'file_name': '000000000139.jpg',
 'height': 426,
 'width': 640,
 'license': None,
 'coco_url': None}

In [92]:
count = 0

for filename in file_names:
  # Extracting image 
  img = get_img(filename)
  img_id = img['id']
  img_w = img['width']
  img_h = img['height']

  # Get Annotations for this image
  img_ann = get_img_ann(img_id)

  if img_ann:
    # Opening file for current image
    file_object = open(f"{output_path}/val_labels/img{count}.txt", "a")

    for ann in img_ann:
      current_category = ann['category_id'] - 1 # As yolo format labels start from 0 
      current_bbox = ann['bbox']
      x = current_bbox[0]
      y = current_bbox[1]
      w = current_bbox[2]
      h = current_bbox[3]
      
      # Finding midpoints
      x_centre = (x + (x+w))/2
      y_centre = (y + (y+h))/2
      
      # Normalization
      x_centre = x_centre / img_w
      y_centre = y_centre / img_h
      w = w / img_w
      h = h / img_h
      
      # Limiting upto fix number of decimal places
      x_centre = format(x_centre, '.6f')
      y_centre = format(y_centre, '.6f')
      w = format(w, '.6f')
      h = format(h, '.6f')
          
      # Writing current object 
      file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")

    file_object.close()
    count += 1  # This should be outside the if img_ann block.

In [93]:
import cv2

def draw_rectangles(image_path, yolo_file_path, output_image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Read the YOLO format text file
    with open(yolo_file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        # Parse YOLO format
        values = line.split()
        class_label = int(values[0])
        x_center = float(values[1])
        y_center = float(values[2])
        width = float(values[3])
        height = float(values[4])

        # Convert YOLO format to OpenCV format
        x = int((x_center - width / 2) * image.shape[1])
        y = int((y_center - height / 2) * image.shape[0])
        w = int(width * image.shape[1])
        h = int(height * image.shape[0])

        # Draw rectangle on the image
        color = (0, 255, 0)  # Green color
        thickness = 2
        image = cv2.rectangle(image, (x, y), (x + w, y + h), color, thickness)

    # Save the output image
    cv2.imwrite(output_image_path, image)
    cv2.imshow("Result", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset_yolo/val_images/img1.jpg'
yolo_file_path = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/dataset_yolo/val_labels/img1.txt'
output_image_path = '/Users/daniil.yefimov/Desktop/GitHub/Autonomous-Vehicle/output_image.jpg'

draw_rectangles(image_path, yolo_file_path, output_image_path)
